# Vorbereitung der Auswertung
Im Folgenden werden die Umfragedaten eingelesen, formatiert und Ausreißer herausgefiltert. Ab der Überschrift '3.2.3 Stichprobe' ist der Inhalt identisch zu gleichnamigem Abschnitt der Arbeit.

## Einlesen und formatieren der Umfragedaten
Die Daten der Umfrage sind unter 'survey_data/survey_raw.csv' zu finden. Der Prolific-Export unter 'survey_data/prolific_raw.csv' enthält weitere Daten über die Probanden. Beide Datensätze werden eingelesen und miteinander verbunden (JOIN). Anschließend werden einige Spalten formatiert, um diese später einfacher auswerten zu können.
Zusätzlich wird die im Skript '0_preparation.ipynb' exportierte Sichprobe der Reviews eingelesen.

In [48]:
import pandas as pd
from scipy.stats import binom
import warnings

# PerformanceWarnings werden unterdrückt in der Ausgabe
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)

# Einlesen und zusammenfügen der Datensätze
df_reviews = pd.read_csv('review_samples/sample_df.csv')
df_survey = pd.read_csv('survey_data/survey_raw.csv', delimiter=';')
df_prolific = pd.read_csv('survey_data/prolific_raw.csv')
df = pd.merge(df_survey, df_prolific, left_on="v_prolific_id", right_on="Participant id", how='outer')

# Daten werden wieder leserlich kodiert
mapping = {1: 'pos', 2: 'neg', -77: 'null'}
v_solo = [f"v_solo_{i}" for i in range(20)]
v_group1 = [f"v_group1_{i}" for i in range(20)]
v_group2 = [f"v_group2_{i}" for i in range(20)]
v_group3 = [f"v_group3_{i}" for i in range(20)]
columns = v_solo + v_group1 + v_group2 + v_group3

for c in columns:
    df[c] = df[c].map(mapping)

mapping = {1: 'male', 2: 'female', 3: 'diverse'}
df['v_gender'] = df['v_gender'].map(mapping)

mapping = {1: 'high school', 2: 'bachelor', 3: 'master', 4: 'doctor'}
df['v_education'] = df['v_education'].map(mapping)

# Berechnen neuer Spalten für spätere Auswertung
for i in range(20):
    # Die KI-assistierten Antworten werden in einheitliche Spalten geschrieben
    df[f"v_ai_{i}"] = df.apply(lambda row: row[f"v_group{row['group']}_{i}"], axis=1)
    # Neue Spalten ob KI-assistierte Antworten richtig waren
    df[f"v_ai_correct_{i}"] = df.apply(lambda row: int(row[f"v_ai_{i}"] == df_reviews.loc[i, 'rating_binary']), axis=1)
    # Neue Spalten ob erste (ohne KI) Antworten richtig waren
    df[f"v_solo_correct_{i}"] = df.apply(lambda row: int(row[f"v_solo_{i}"] == df_reviews.loc[i, 'rating_binary']), axis=1)
    # Neue Spalten ob Antwort geändert wurde
    df[f"v_changed_{i}"] = df.apply(lambda row: int(row[f"v_solo_{i}"] != row[f"v_ai_{i}"]), axis=1)
    # Neue Spalten ob Antwort geändert und an KI angepasst wurde
    df[f"v_followed_ai_{i}"] = df.apply(lambda row: int((row[f"v_solo_{i}"] != row[f"v_ai_{i}"]) & (row[f"v_ai_{i}"] == df_reviews.loc[i, 'predictions'])), axis=1)
    # Neue Spalten ob Antwort geändert und an KI angepasst wurde, wenn KI richtig lag
    df[f"v_followed_correct_ai_{i}"] = df.apply(lambda row: int((row[f"v_solo_{i}"] != row[f"v_ai_{i}"]) & (row[f"v_ai_{i}"] == df_reviews.loc[i, 'predictions']) & (df_reviews.loc[i, 'predictions'] == df_reviews.loc[i, 'rating_binary'])), axis=1)
    # Neue Spalten ob Antwort geändert und an KI angepasst wurde, wenn KI falsch lag
    df[f"v_followed_wrong_ai_{i}"] = df.apply(lambda row: int((row[f"v_solo_{i}"] != row[f"v_ai_{i}"]) & (row[f"v_ai_{i}"] == df_reviews.loc[i, 'predictions']) & (df_reviews.loc[i, 'predictions'] != df_reviews.loc[i, 'rating_binary'])), axis=1)
    # Neue Spalten ob zweite Antwort gleich der KI Einschätzung ist
    df[f"v_same_as_ai_{i}"] = df.apply(lambda row: int(row[f"v_ai_{i}"] == df_reviews.loc[i, 'predictions']), axis=1)


# Berechnung der Gesamtanzahlen
df['v_ai_correct_sum'] = df.filter(like='v_ai_correct').sum(axis=1)
df['v_solo_correct_sum'] = df.filter(like='v_solo_correct').sum(axis=1)
df['v_changed_sum'] = df.filter(like='v_changed').sum(axis=1)
df['v_followed_ai_sum'] = df.filter(like='v_followed_ai').sum(axis=1)
df['v_followed_correct_ai_sum'] = df.filter(like='v_followed_correct_ai').sum(axis=1)
df['v_followed_wrong_ai_sum'] = df.filter(like='v_followed_wrong_ai').sum(axis=1)
df['v_same_as_ai_sum'] = df.filter(like='v_same_as_ai').sum(axis=1)

df

,lfdn,duration,group,c_0002,v_age,v_gender,v_education,v_prolific_id,v_birth_year,v_solo_0,...,v_followed_correct_ai_19,v_followed_wrong_ai_19,v_same_as_ai_19,v_ai_correct_sum,v_solo_correct_sum,v_changed_sum,v_followed_ai_sum,v_followed_correct_ai_sum,v_followed_wrong_ai_sum,v_same_as_ai_sum
0,10,889,2,82401,41,male,high school,5ab15ab4c42b630001cd36fb,1982,pos,...,0,0,1,19,19,0,0,0,0,18
1,14,1286,1,82687,35,male,doctor,5680a047c5767f00121cca5e,1988,pos,...,0,0,1,19,19,0,0,0,0,18
2,13,766,3,82995,29,male,high school,5f733312e59fd2000a74b8cf,1994,pos,...,0,0,1,16,18,2,0,0,0,17
3,21,966,1,81095,26,female,bachelor,6170909684361d99aca08169,1997,pos,...,0,0,1,19,19,0,0,0,0,18
4,19,1224,3,82943,22,diverse,bachelor,5d1cc1adb50874001a5792c8,2000,pos,...,0,0,1,20,20,0,0,0,0,17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78,96,981,3,84999,36,female,bachelor,611a78efa6c36a674dc40cc6,1987,pos,...,0,0,1,20,20,0,0,0,0,17
79,95,881,1,85000,35,male,master,5f913cbf314c3d000982e130,1988,pos,...,0,0,1,19,20,1,1,0,1,18
80,94,985,2,89824,38,female,bachelor,5f2f62bb956a3f34b614a33f,1985,pos,...,0,0,1,19,19,0,0,0,0,18
81,92,1363,2,86443,21,female,high school,64933010b2c7da9cc8cfc175,2002,pos,...,0,0,1,19,19,0,0,0,0,18


## Berechnung der Bonuszahlungen
Gemäß der Ausschreibung wurde allen Probanden mit einer Performance über Median ein Bonus gewährt. Dazu wurde eine Datei geschrieben, die in Prolific importiert wurde. Der Median lag bei 19 richtig eingeschätzten Reviews. Insgesamt lagen 43 Probanden über dem Schwellwert.

In [49]:
df_bonus_payment = df[['v_prolific_id', 'Participant id', 'v_ai_correct_sum', 'v_solo_correct_sum']].copy()
df_bonus_payment = df_bonus_payment[df_bonus_payment['v_ai_correct_sum'] >= 19]

formatted_data = df_bonus_payment.apply(lambda row: f"{row['v_prolific_id']},1.00", axis=1)
with open("survey_data/prolific_bonus_payments.txt", "w") as file:
    for entry in formatted_data:
        file.write(entry + "\n")

len(df_bonus_payment)

43

## 3.2.3 Stichprobe
An der Studie nahmen insgesamt 83 Probanden teil. Die Veränderung zu den 80 Probanden, die in Prolific ausgeschrieben waren, ergibt sich dadurch, dass eine Teilnahme zurückgewiesen wurde und somit ein Ersatz von Prolific zur Verfügung gestellt wurde. Außerdem wurden zwei Probanden, deren Teilnahme in Prolific fälschlicherweise als nicht vollständig markiert wurde, nachträglich akzeptiert. Um Probanden auszusortieren, die die Umfrage offensichtlich nicht richtig ausgefüllt hatten, wurden zwei verschiedene Tests gemacht und dabei jeweils im Einzelfall bewertet.

Der erste Test war der in der Umfrage eingebaute Aufmerksamkeitstest. Zu Beginn der Umfrage sollten die Probanden ihr Alter angeben, zum Ende ihr Geburtsjahr. Bei drei Probanden wurden dabei falsche Werte eingetragen:

In [50]:
outliers = []

df_age_test = df[[ 'v_age', 'v_birth_year', 'Age', 'v_prolific_id']].copy()
df_age_test['Sum_of_Age_and_BirthYear'] = df_age_test['v_age'] + df_age_test['v_birth_year']

df_age_test = df_age_test[(df_age_test['Sum_of_Age_and_BirthYear'] != 2023) & (df_age_test['Sum_of_Age_and_BirthYear'] != 2022)]

outliers = [*outliers, 50, 64]

df_age_test

,v_age,v_birth_year,Age,v_prolific_id,Sum_of_Age_and_BirthYear
50,58,1957,CONSENT_REVOKED,5eaa9d9c107b78017fab3390,2015
55,34,1987,35,614dc6a01b4262b98ffa6d22,2021
64,37,36,37,60d2f9397932f9695f49f5f4,73


Die Probanden mit den IDs 50 und 64 wurden aus der Stichprobe entfernt. Bei dem Teilnehmer mit der ID 55 ist davon auszugehen, dass dieser lediglich versehentlich ein falsches Alter angegeben hat. Dieser wurde daher in der Stichprobe belassen.

Der zweite Test untersuchte ob Probanden auffällig oft ihre Einschätzung gegen die KI-Empfehlung geändert hatten. Dies könnte ein Zeichen dafür sein, dass Teilnehmer die Reviews nicht lesen und auf jeder Seite etwas anderes ankreuzen. Als zusätzlicher Faktor wurde dazu die Performance der Probanden betrachtet. Nur zwei Probanden wechselten gegen den KI-Vorschlag:

In [51]:
df_change_against_ai = df[['v_solo_correct_sum', 'v_ai_correct_sum', 'v_prolific_id',  'v_changed_sum', 'v_followed_ai_sum']].copy()
df_change_against_ai['change_against_ai'] = df_change_against_ai['v_changed_sum'] - df_change_against_ai['v_followed_ai_sum']
outliers = [*outliers, 69]

df_change_against_ai[df_change_against_ai['change_against_ai'] >= 2]

,v_solo_correct_sum,v_ai_correct_sum,v_prolific_id,v_changed_sum,v_followed_ai_sum,change_against_ai
2,18,16,5f733312e59fd2000a74b8cf,2,0,2
69,10,11,609d98a41cdc16106fa64874,5,3,2


Mit einer Solo-Performance von 18 beziehungsweise 90% kann davon ausgegangen werden, dass der Proband mit der ID 2 die Fragen ordnungsgemäß beantwortet hat. Dies ist bei dem Probanden 69 nicht der Fall mit einer Solo-Performance von 50% (10). Daher wird dieser aus der Stichprobe entfernt.

In [52]:
outliers

[50, 64, 69]

In [53]:
df = df.drop(outliers)
df.to_csv('survey_data/survey_cleaned.csv', index=False)

df

,lfdn,duration,group,c_0002,v_age,v_gender,v_education,v_prolific_id,v_birth_year,v_solo_0,...,v_followed_correct_ai_19,v_followed_wrong_ai_19,v_same_as_ai_19,v_ai_correct_sum,v_solo_correct_sum,v_changed_sum,v_followed_ai_sum,v_followed_correct_ai_sum,v_followed_wrong_ai_sum,v_same_as_ai_sum
0,10,889,2,82401,41,male,high school,5ab15ab4c42b630001cd36fb,1982,pos,...,0,0,1,19,19,0,0,0,0,18
1,14,1286,1,82687,35,male,doctor,5680a047c5767f00121cca5e,1988,pos,...,0,0,1,19,19,0,0,0,0,18
2,13,766,3,82995,29,male,high school,5f733312e59fd2000a74b8cf,1994,pos,...,0,0,1,16,18,2,0,0,0,17
3,21,966,1,81095,26,female,bachelor,6170909684361d99aca08169,1997,pos,...,0,0,1,19,19,0,0,0,0,18
4,19,1224,3,82943,22,diverse,bachelor,5d1cc1adb50874001a5792c8,2000,pos,...,0,0,1,20,20,0,0,0,0,17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78,96,981,3,84999,36,female,bachelor,611a78efa6c36a674dc40cc6,1987,pos,...,0,0,1,20,20,0,0,0,0,17
79,95,881,1,85000,35,male,master,5f913cbf314c3d000982e130,1988,pos,...,0,0,1,19,20,1,1,0,1,18
80,94,985,2,89824,38,female,bachelor,5f2f62bb956a3f34b614a33f,1985,pos,...,0,0,1,19,19,0,0,0,0,18
81,92,1363,2,86443,21,female,high school,64933010b2c7da9cc8cfc175,2002,pos,...,0,0,1,19,19,0,0,0,0,18


Von den 83 Teilnehmern wurden somit 3 entfernt, wodurch sich die Stichprobengröße von 80 Probanden ergibt. Die Verteilung hinsichtlich der Gruppen, Gender, Bildungsgrad, Alter und Nationalität kann den folgenden Tabellen entnommen werden:

In [54]:
def compute_distribution_and_percent_for_column(dataframe, column_name):
    distribution = dataframe[column_name].value_counts().reset_index()
    distribution.columns = ['Kategorie', 'Anzahl']
    distribution['Prozent'] = (distribution['Anzahl'] / distribution['Anzahl'].sum()) * 100
    return distribution

# Altersgruppen erstellen
bins = [18, 25, 35, 45, 55, 65, 100]
labels = ['18-24', '25-34', '35-44', '45-54', '55-64', '65+']
df['age_group'] = pd.cut(df['v_age'], bins=bins, labels=labels, right=False)

# Verteilungen mit Prozentangaben berechnen
group_distribution_code = compute_distribution_and_percent_for_column(df, 'group')
gender_distribution_code = compute_distribution_and_percent_for_column(df, 'v_gender')
education_distribution_code = compute_distribution_and_percent_for_column(df, 'v_education')
age_distribution_code = compute_distribution_and_percent_for_column(df, 'age_group')
nationality_distribution_code = compute_distribution_and_percent_for_column(df, 'Nationality')

display(group_distribution_code, gender_distribution_code, education_distribution_code, age_distribution_code, nationality_distribution_code)

,Kategorie,Anzahl,Prozent
0,3,29,36.25
1,1,28,35.00
2,2,23,28.75


,Kategorie,Anzahl,Prozent
0,male,39,48.75
1,female,39,48.75
2,diverse,2,2.50


,Kategorie,Anzahl,Prozent
0,bachelor,43,53.75
1,high school,22,27.50
2,master,14,17.50
3,doctor,1,1.25


,Kategorie,Anzahl,Prozent
0,18-24,36,45.00
1,25-34,30,37.50
2,35-44,12,15.00
3,45-54,1,1.25
4,55-64,1,1.25
5,65+,0,0.00


,Kategorie,Anzahl,Prozent
0,Poland,15,18.75
1,Portugal,12,15.00
2,South Africa,12,15.00
3,Mexico,10,12.50
4,United Kingdom,6,7.50
5,Hungary,4,5.00
6,Italy,4,5.00
7,United States,4,5.00
8,Greece,3,3.75
9,Spain,3,3.75
